### Importing Libraries

In [ ]:
import math
import numpy as np
import pandas as pd
import time
from sklearn.svm import SVC
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import cv2
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

### Importing Dataset

In [ ]:
df = pd.read_csv('/kaggle/input/fer2013/fer2013.csv')

In [ ]:
df.head()

In [ ]:
emotion_label_to_text = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}

In [ ]:
df.emotion.value_counts()

### Visualize Dataset

In [ ]:
fig = plt.figure(1, (14, 14))

k = 0
for label in sorted(df.emotion.unique()):
    for j in range(1):
        px = df[df.emotion==label].pixels.iloc[k]
        px = np.array(px.split(' ')).reshape(48, 48).astype('float32')

        k += 1
        ax = plt.subplot(7, 7, k)
        ax.imshow(px , cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(emotion_label_to_text[label])
        plt.tight_layout()

### Input and Output Data

In [ ]:
X = df.pixels.apply(lambda x: np.array(x.split(' ')).astype('float32'))
X = np.stack(X, axis=0)
Y = np.array(df['emotion'])

In [ ]:
X.shape

In [ ]:
Y.shape

# Preprocessing
Face Alignment 

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

In [ ]:
def face_align(gray):
  sz = gray.shape
  # Creating variable eyes
  eyes = eye_cascade.detectMultiScale(gray, 1.3, 5)
  if len(eyes) > 1 :
    index=0
    # Creating for loop in order to divide one eye from another
    for (ex , ey,  ew,  eh) in eyes:
      if index == 0:
        eye_1 = (ex, ey, ew, eh)
      elif index == 1:
        eye_2 = (ex, ey, ew, eh)
      index += 1
    if eye_1[0] < eye_2[0]:
      left_eye = eye_1
      right_eye = eye_2
    else:
      left_eye = eye_2
      right_eye = eye_1
        
    # Calculating coordinates of a central points of the rectangles
    left_eye_center = (int(left_eye[0] + (left_eye[2] / 2)), int(left_eye[1] + (left_eye[3] / 2)))
    left_eye_x = left_eye_center[0] 
    left_eye_y = left_eye_center[1]
        
    right_eye_center = (int(right_eye[0] + (right_eye[2]/2)), int(right_eye[1] + (right_eye[3]/2)))
    right_eye_x = right_eye_center[0]
    right_eye_y = right_eye_center[1]

    if left_eye_y > right_eye_y:
      A = (right_eye_x, left_eye_y)
      # Integer -1 indicates that the image will rotate in the clockwise direction
      direction = -1 
    else:
      A = (left_eye_x, right_eye_y)
      # Integer 1 indicates that image will rotate in the counter clockwise direction
      direction = 1 
        
    delta_x = right_eye_x - left_eye_x
    delta_y = right_eye_y - left_eye_y
    angle=np.arctan(delta_y/delta_x)
    angle = (angle * 180) / np.pi

    # Width and height of the image
    h, w = gray.shape[:2]
    # Calculating a center point of the image
    # Integer division "//"" ensures that we receive whole numbers
    center = (w // 2, h // 2)
    # Defining a matrix M and calling
    # cv2.getRotationMatrix2D method
    M = cv2.getRotationMatrix2D(center, (angle), 1.0)
    # Applying the rotation to our image using the
    # cv2.warpAffine method
    gray = cv2.warpAffine(gray, M, (w, h))

    #Again detecting face
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    if len(faces) > 0 : 
      (x,y,w,h) = faces[0]
      gray = gray[y:y+h, x:x+w]

  gray = cv2.resize(gray,sz) 
  return gray

In [ ]:
X_new = []
for j in range(X.shape[0]):
  px = np.array(X[j]).reshape(48,48)
  px = np.array(px, dtype='uint8')
  px = face_align(px)
  X_new.append(np.array(px).reshape(48*48))
X = np.array(X_new)

#### Confusion Matrix

In [ ]:
# Plot confusion matrix 

import seaborn as sns
import matplotlib.pyplot as plt

def create_confmat(true_labels, predicted_labels, columns, colour = 'Greens', size = (20,14)):
    sns.set(font_scale=1.5)
    cm = confusion_matrix(true_labels, predicted_labels) 
    cm_df = pd.DataFrame(cm,
    index = [col for col in columns], 
    columns = [col for col in columns])
    plt.figure(figsize=(18,16))
    sns.heatmap(cm_df, annot = True, cmap = colour, fmt='g', linewidths=.2)
    plt.title('Confusion Matrix', fontsize = 20)
    plt.ylabel('True label', fontsize = 18)
    plt.xlabel('Predicted label', fontsize = 18)
    plt.tick_params(axis='both', labelsize=14)
    plt.show()

### Train and Test Split for KNN and SVM


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, Y,
                                                    shuffle=True, stratify=Y,
                                                    test_size=0.2, random_state=2)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

###PCA to retain 95% variation and reduce dataset size

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.astype(np.float32))
X_valid = scaler.transform(X_valid.astype(np.float32))
pca = PCA(n_components=.95)
pca.fit(X_train)
X_test_p = pca.transform(X_valid)
X_train_p = pca.transform(X_train)
scaler = StandardScaler()
X_train_p = scaler.fit_transform(X_train_p.astype(np.float32))
X_test_p = scaler.transform(X_test_p.astype(np.float32))
X_train_p.shape ,X_test_p.shape 


# SVM

####Using Linear kernel

In [ ]:
model = SVC(kernel = 'linear')
model.fit(X_train_p,y_train)
model.score(X_test_p , y_valid)

####Using rbf kernel

In [ ]:
model = SVC(kernel ='rbf')
model.fit(X_train_p,y_train)
model.score(X_test_p , y_valid)

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=1)
classifier.fit(X_train, y_train)
classifier.score(X_valid,y_valid)

In [ ]:
start_time = time.time()
y_pred = classifier.predict(X_valid)
print("Time Taken : %s seconds " % (time.time() - start_time))

In [ ]:
cnt = 0
for i in range(X_valid.shape[0]):
   cnt += y_pred[i] == y_valid[i]
print("Accuracy :", cnt/X_valid.shape[0]*100, "%")

####KNN using default parameters

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train_p ,y_train)
knn.score(X_test_p,y_valid)